In [11]:
#Python bash script to find all the strings in a executable 
import os
os.system('for i in `ls TestFiles/ | xargs` ; do `strings TestFiles/$i > textFiles/$i.txt`; done')

0

In [10]:
#Now these text files needs to be analyzed

In [12]:
#All the imports for the program go here
import time
import sys
from os import chdir
from os import getcwd
from os import path
import os
import math
from collections import Counter
import numpy as np 
import tensorflow as tf
from ktensor import KruskalTensor
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from sklearn.metrics.cluster import v_measure_score
from scipy import spatial

In [30]:
# This block takes the present working directory 
oldwd = getcwd()
#dataDir = oldwd+"/LZsetUpdated/zeusBinary/"
#dataDir = oldwd+"/TermDocumentTensor/test_files/"
dataDir = "/Users/Phaniteja/desktop/Github/Tensor-Decomposition/textFiles/"
print(oldwd)
text_files = [f for f in os.listdir(dataDir)]# if f.endswith('.exe')]
numberOfFiles = len(text_files)
print("Found %d files to look at" % numberOfFiles)

/Users/Phaniteja/Desktop/Github/Tensor-Decomposition
Found 191 files to look at


In [16]:
malwareCount=[]
benignCount= []
for i in range(0,len(text_files)):
    if text_files[i][0:4]=="zeus":
        malwareCount.append(text_files[i])
    else:
        benignCount.append(text_files[i])

In [18]:
print(len(benignCount),len(malwareCount))

121 70


In [19]:
#All the initilizations in the project 
unpacked_files = []
count = 0
mostFrequentNgrams = []
length_Ngram_Dictionaries = []
global_Dictionary_ngrams = {}
file_Dictionary_ngram = []

In [20]:
#definition for the accuracy of the given dataset
def findAccuracies(groundTruth,predictedLabel):
    real_Label = []
    for i in range(0,len(groundTruth)):
        if groundTruth[i][:4]=="zeus":
            real_Label.append(1)
        else:
            real_Label.append(0)
    print((real_Label),(predictedLabel))
    return v_measure_score(ground_Truth,list(Predicted_Label))*100

In [21]:
#Defining entropy
def entropy(s):
    p, lns = Counter(s), float(len(s))
    return -sum( count/lns * math.log(count/lns, 2) 
                for count in p.values())

In [22]:
#creates the ngram dictionary and returns it
def createTensor(file,ngram):
    #create the ngrams first and then add them in the dictionary 
    nGram_dictionary = {}
    for i in range(0,len(file),ngram):
        ngram_word=file[i:i+ngram]
        if ngram_word not in global_Dictionary_ngrams:
            global_Dictionary_ngrams[ngram_word]=1
        if ngram_word in nGram_dictionary:
            nGram_dictionary[ngram_word][0]+=1
            nGram_dictionary[ngram_word][1].append(location_Calculator(i,len(file)))
        else:
            nGram_dictionary[ngram_word]=[]
            nGram_dictionary[ngram_word].append(1)
            loc = location_Calculator(i,len(file))
            nGram_dictionary[ngram_word].append([loc])  
    return nGram_dictionary 

In [23]:
#given the file length and its ngram it returns the location in file
def location_Calculator(i,length):
    location_word = i/length*100
    #print(location_word)
    if location_word <=10:
        return 10
    elif location_word <=20:
        return 20
    elif location_word <=30:
        return 30
    elif location_word <=40:
        return 40
    elif location_word <=50:
        return 50
    elif location_word <=60:
        return 60
    elif location_word <=70:
        return 70
    elif location_word <=80:
        return 80
    elif location_word <=90:
        return 90
    return 100

In [24]:
#creates the tensor from the given dictionaries
def makeATensor(globalDict,fileDict):
    #print(globalDict,fileDict)
    tensor = np.zeros((numberOfFiles,len(globalDict),11))
    count_file = 0
    total_Ngrams = 0 
    for f in text_files:
        theFile = open(dataDir+f, "rb").read()
        theFile = theFile[:1000]
        for i in range(0,len(theFile),5):
            total_Ngrams+=1
            ngram_word=theFile[i:i+5]
            index_File = count_file
            index_Ngram = globalDict[ngram_word]
            loc_Ngram = location_Calculator(i,len(theFile))
            #print(index_File,index_Ngram,int(loc_Ngram/10))
            tensor[index_File][index_Ngram][int(loc_Ngram/10)]+=1    
        count_file +=1
    file = open("tensorTesting.txt","w") 
    file.write(str(tensor))
    file.close() 
#     print(tensor[1])
#     #x = np.arange(200).reshape((4,5,10))
#     with file('test.txt', 'w') as outfile:
#         for slice_2d in tensor:
#             np.savetxt(outfile, slice_2d)
    print("Ngrams are")
    print(total_Ngrams)
    return tensor

In [25]:
#decomposition Tecnique for the given tensor
def parafac_decomposition(tensor):
        """
        Computes a parafac decomposition of the tensor.
        This will return n rank 3 factor matrices, where n represents the 
        dimensionality of the tensor.
        :return:
        """
        decompose = KruskalTensor(tensor.shape, rank=3, regularize=1e-6, 
                                  init='nvecs', X_data=tensor)
        factors = decompose.U
        with tf.Session() as sess:
            for i in range(len(factors)):
                sess.run(factors[i].initializer)
                factors[i] = factors[i].eval()
        return factors

In [26]:
#for vizualizing and its clustering we use this function 
def k_means_clustering(factor_matrix):
        clusters = 2
        svd = TruncatedSVD(n_components=2, n_iter=20, random_state=42)
        reduced = svd.fit_transform(factor_matrix)
        kmeans = KMeans(n_clusters=clusters, random_state=0).fit(factor_matrix)
        labels_predicted = kmeans.labels_
        #print(labels_predicted)
        #print(text_files)
        #lets write the function to check the accuracy here 
        #we have 4 different categories 0-zeus 0-benign 1-zeus 0-benign
        zero_Zeus = 0
        zero_Benign = 0
        one_Zeus = 0 
        one_Benign = 0 
        for i in range(0,len(text_files)):
            #print(text_files[i][len(text_files[i])-3:],labels_predicted[i])
            if text_files[i][len(text_files[i])-3:]=="exe" and labels_predicted[i]==0:
                zero_Benign+=1
            elif text_files[i][len(text_files[i])-3:]=="exe" and labels_predicted[i]==1:
                one_Benign+=1
            elif text_files[i][len(text_files[i])-3:]=="ex0" and labels_predicted[i]==0:
                zero_Zeus+=1
            elif text_files[i][len(text_files[i])-3:]=="ex0" and labels_predicted[i]==1:
                one_Zeus+=1
            else:
                continue
        print(zero_Zeus,zero_Benign,one_Zeus,one_Benign) 
        data = [plotly.graph_objs.Scatter(x=[entry[0] for entry in reduced],
                                          y=[entry[1] for entry in reduced],
                                          mode='markers',
                                          marker=dict(color=kmeans.labels_),
                                          text = text_files
                                          )
                ]
        fig = go.Figure(data=data)
        plotly.offline.plot(fig, filename='kmeans_cluster.html')
        return text_files,labels_predicted

In [27]:
#function to create the similarity matrix for the heatmap used
def generate_cosine_similarity_matrix(matrix):
    cosine_sim = []
    for entry in matrix:
        sim = []
        for other_entry in matrix:
            sim.append(spatial.distance.cosine(entry, other_entry) * -1 + 1)
        cosine_sim.append(sim)
    return cosine_sim

In [28]:
#function to create the heatmap of the following tensor decomposition
def generate_heat_map(data, axis_labels):
    """
    Generates a heat map for the current data
    Currently only meant to support using a cosine similarity matrix
    :param data:
    :param axis_labels:
    :return:
    """
    axis_labels_abbreviated = [label[:14] for label in axis_labels]
    info = [go.Heatmap(z=data,
                       x=axis_labels_abbreviated,
                       y=axis_labels_abbreviated,
                       colorscale='Hot',
                       )]

    layout = go.Layout(title='Cosine Similarity Between Documents',
                       xaxis=dict(ticks=''),
                       yaxis=dict(ticks=''),
                       plot_bgcolor='#444',
                       paper_bgcolor='#eee'
                       )
    fig = go.Figure(data=info, layout=layout)
    plotly.offline.plot(fig, filename='heatmap.html')

In [31]:
#This the clock where we run the whole program and its the driver program!
for f in text_files:
    count+=1
    if count == 250:
        print("read file ",f)
        count = 0
    theFile = open(dataDir+f, "rb").read()
    #print(len(theFile))
    #call the tensor decomposition stuff here and create the tensor here
    dictNgrams_file = createTensor(theFile[:1000],5)
    file_Dictionary_ngram.append(dictNgrams_file)
    #print(dictNgrams_file)
    length_Ngram_Dictionaries.append(len(dictNgrams_file))
    for key in dictNgrams_file:
        if dictNgrams_file[key][0]>1:
            mostFrequentNgrams.append([key,dictNgrams_file[key]])
    #print(mostFrequentNgrams)
    #print("Next Exe File")
#print(max(length_Ngram_Dictionaries))
j = 0
for key in global_Dictionary_ngrams:
    global_Dictionary_ngrams[key]=j
    j = j+1    
#call a function which takes the global dictionary, each file dictionary and creates the tensor
tensor_Ngram = makeATensor(global_Dictionary_ngrams,file_Dictionary_ngram)
#print(global_Dictionary_ngrams)   
#print(tensor_Ngram)
#print(np.amax(tensor_Ngram))    
#print(np.sum(tensor_Ngram))
#Got the tensor here as tensor_Ngram, now lets form its factor matrices 
#and then try to cluster them 
factors_Ngrams = parafac_decomposition(tensor_Ngram)
#print(factors_Ngrams[2])
#lets send the factor matrices to k means clustering to see how the files will be clustered

k_means_clustering(factors_Ngrams[0])
#here we are done with the ploting 
# let's check how correct we are goin in the current process
#Run next cell for the accuracies
cos_sim = generate_cosine_similarity_matrix(factors_Ngrams[0])
generate_heat_map(cos_sim, text_files)

# if args['heatmap']:
#         cos_sim = tdt.generate_cosine_similarity_matrix(factors[args['axis']])
#         visualize.generate_heat_map(cos_sim, tdt.corpus_names)
#     if args['kmeans']:
#         visualize.k_means_clustering(factors[args['axis']], tdt.corpus_names, 
#                                      clusters=args['components'])

Ngrams are
37456
0 0 0 0
